In [1]:
#diabetes df
import pandas as pd
import numpy as np
import pandas_profiling

directory = 'C:/githubrepo/7331_Project/data/'
df_zg = pd.read_csv(directory + 'diabetic_data.csv')
df_zg.replace('?', np.nan, inplace = True)

In [2]:
df_zg.shape

(101766, 50)

In [3]:
# Data Distribution
df_zg.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.675904,4.388362,4.372443,4.589696,34.047383,1.456577,16.560492,0.369357,0.212232,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.530799,6.013795,4.687549,3.160314,22.386944,1.792201,9.105980,1.267265,0.917842,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,16.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,3.000000,1.000000,2.000000,4.000000,30.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,4.000000,6.000000,7.000000,6.000000,49.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,129.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [4]:
missing = pd.DataFrame({'column':df_zg.columns ,'na_percent':df_zg.isnull().sum()/len(df_zg)*100, 'na_count':df_zg.isnull().sum()})
missing[missing.na_percent > 0]

,column,na_percent,na_count
race,race,2.233555,2273
weight,weight,96.858479,98569
payer_code,payer_code,39.557416,40256
medical_specialty,medical_specialty,49.082208,49949
diag_1,diag_1,0.020636,21
diag_2,diag_2,0.351787,358
diag_3,diag_3,1.398306,1423


### Fixing race, weight, payer_code, medical_speciality, examide, and citoglipton

In [5]:
#Remove Weight and payer code because they have too many NA's
#Remove examide and citoglipton because they are categorical with only 1 category
df_zg.drop(['weight', 'payer_code', 'examide', 'citoglipton'], axis=1, inplace = True)

#Fix NA's in medical speciality
df_zg.loc[df_zg['medical_specialty'].isnull(), 'medical_specialty'] = "Unspecified"
#Fix race NA's
df_zg.loc[df_zg['race'].isnull(), 'race'] = "Unspecified"

### Checking the number of NA's again

In [6]:
missing = pd.DataFrame({'column':df_zg.columns ,'na_percent':df_zg.isnull().sum()/len(df_zg)*100, 'na_count':df_zg.isnull().sum()})
missing[missing.na_percent > 0]

,column,na_percent,na_count
diag_1,diag_1,0.020636,21
diag_2,diag_2,0.351787,358
diag_3,diag_3,1.398306,1423


##### Prepping to fix diag_1,2,3. Because the codes starting with E and V's relate to something specific just like the others that are numeric, we are changing them to be numeric so that we can replace them with meaningful categories

In [7]:
#The 3 columns to fix
d_nums = ['diag_1', 'diag_2', 'diag_3']

#Removing the Letter at the start with a number
df_zg[d_nums] = df_zg[d_nums].replace({'E':'-1'}, regex = True)
df_zg[d_nums] = df_zg[d_nums].replace({'V':'-2'}, regex = True)

#Changing it to be numeric
df_zg[d_nums] = df_zg[d_nums].astype(np.float)

##### Fixing diag_1

In [8]:
#Supress all warnings
pd.options.mode.chained_assignment = None

#Function to fix the three diag columns
def fixdiags(name, newname):
    #Website with the codes
    #http://www.icd9data.com/2012/Volume1/780-799/780-789/default.htm
    #First creating an empty column so that we don't mess with the original
    df_zg[newname] = np.nan #empty column

    #Making the categories based on the numerical value
    df_zg[newname][(df_zg[name] >= -2000) & (df_zg[name] < 1)] = "Other"
    df_zg[newname][(df_zg[name] >= 1) & (df_zg[name] < 250)] = "Neoplasms"
    df_zg[newname][(df_zg[name] >= 250) & (df_zg[name] < 251)] = "Diabeties"
    df_zg[newname][(df_zg[name] >= 251) & (df_zg[name] < 280)] = "Neoplasms"
    df_zg[newname][(df_zg[name] >= 280) & (df_zg[name] <= 389)] = "Other"
    df_zg[newname][((df_zg[name] >= 390) & (df_zg[name] <= 459)) | (df_zg[name] == 785)] = "Circulatory"
    df_zg[newname][((df_zg[name] >= 460) & (df_zg[name] <= 519)) | (df_zg[name] == 786)] = "Respiratory"
    df_zg[newname][((df_zg[name] >= 520) & (df_zg[name] <= 579)) | (df_zg[name] == 787)] = "Digestive"
    df_zg[newname][((df_zg[name] >= 580) & (df_zg[name] <= 629)) | (df_zg[name] == 788)] = "Genitourinary"
    df_zg[newname][(df_zg[name] >= 630) & (df_zg[name] <= 679)] = "Other"
    df_zg[newname][((df_zg[name] >= 680) & (df_zg[name] <= 709)) | (df_zg[name] == 782)] = "Neoplasms"
    df_zg[newname][(df_zg[name] >= 710) & (df_zg[name] <= 739)] = "Musculoskeletal"
    df_zg[newname][(df_zg[name] >= 740) & (df_zg[name] <= 759)] = "Other"
    df_zg[newname][(df_zg[name] == 783)] = "Digestive"
    df_zg[newname][(df_zg[name] == 789)] = "Other"
    df_zg[newname][((df_zg[name] >= 790) & (df_zg[name] <= 799)) | (df_zg[name] == 780) | (df_zg[name] == 781) | (df_zg[name] == 784)] = "Neoplasms"
    df_zg[newname][(df_zg[name] >= 800) & (df_zg[name] <= 999)] = "Injury"

    #Sets the value of the null diagnosis to None
    df_zg[newname][df_zg[name].isnull()] = "None"
    return df_zg

#Actually running the code to fix diag_1, 2, and 3
df_zg = fixdiags('diag_1', 'diag_1_val')
df_zg = fixdiags('diag_2', 'diag_2_val')
df_zg = fixdiags('diag_3', 'diag_3_val')

#Drop the three columns that are now unneeded
d_nums = ['diag_1', 'diag_2', 'diag_3']
df_zg.drop(d_nums, axis=1, inplace = True)

### Checking NA's again

In [9]:
missing = pd.DataFrame({'column':df_zg.columns ,'na_percent':df_zg.isnull().sum()/len(df_zg)*100, 'na_count':df_zg.isnull().sum()})
missing[missing.na_percent > 0]

,column,na_percent,na_count


### Adding the Readmitted Yes/No (1/0) column

In [10]:
#Supress all warnings
pd.options.mode.chained_assignment = None

#Make a new column for this value
df_zg['readmitted_tf'] = np.nan

#Set the values
df_zg.loc[df_zg['readmitted'] == "NO", 'readmitted_tf'] = "0"
df_zg.loc[df_zg['readmitted'] == "<30", 'readmitted_tf'] = "1"
df_zg.loc[df_zg['readmitted'] == ">30", 'readmitted_tf'] = "1"

#Set the type that it needs to be (integer)
df_zg["readmitted_tf"]=df_zg["readmitted_tf"].astype('int64')

### Gender has only 3 set to unassigned, so we need to impute it. Most common is Female

In [11]:
df_zg['gender'] = df_zg['gender'].replace(to_replace = 'Unknown/Invalid', value = 'Female')

### Making the values 1/0 so that we can easily get a count of how many meds each row is taking

In [12]:
df_zg['metformin-rosiglitazone_tf'] = 1
df_zg['metformin-rosiglitazone_tf'][df_zg["metformin-rosiglitazone"] == "No"] = 0
df_zg['metformin-pioglitazone_tf'] = 1
df_zg['metformin-pioglitazone_tf'][df_zg["metformin-pioglitazone"] == "No"] = 0
df_zg['glimepiride-pioglitazone_tf'] = 1
df_zg['glimepiride-pioglitazone_tf'][df_zg["glimepiride-pioglitazone"] == "No"] = 0
df_zg['glipizide-metformin_tf'] = 1
df_zg['glipizide-metformin_tf'][df_zg["glipizide-metformin"] == "No"] = 0
df_zg['glyburide-metformin_tf'] = 1
df_zg['glyburide-metformin_tf'][df_zg["glyburide-metformin"] == "No"] = 0
df_zg['insulin_tf'] = 1
df_zg['insulin_tf'][df_zg["insulin"] == "No"] = 0
df_zg['tolazamide_tf'] = 1
df_zg['tolazamide_tf'][df_zg["tolazamide"] == "No"] = 0
df_zg['troglitazone_tf'] = 1
df_zg['troglitazone_tf'][df_zg["troglitazone"] == "No"] = 0
df_zg['miglitol_tf'] = 1
df_zg['miglitol_tf'][df_zg["miglitol"] == "No"] = 0
df_zg['acarbose_tf'] = 1
df_zg['acarbose_tf'][df_zg["acarbose"] == "No"] = 0
df_zg['rosiglitazone_tf'] = 1
df_zg['rosiglitazone_tf'][df_zg["rosiglitazone"] == "No"] = 0
df_zg['pioglitazone_tf'] = 1
df_zg['pioglitazone_tf'][df_zg["pioglitazone"] == "No"] = 0
df_zg['metformin_tf'] = 1
df_zg['metformin_tf'][df_zg["metformin"] == "No"] = 0
df_zg['repaglinide_tf'] = 1
df_zg['repaglinide_tf'][df_zg["repaglinide"] == "No"] = 0
df_zg['nateglinide_tf'] = 1
df_zg['nateglinide_tf'][df_zg["nateglinide"] == "No"] = 0
df_zg['chlorpropamide_tf'] = 1
df_zg['chlorpropamide_tf'][df_zg["chlorpropamide"] == "No"] = 0
df_zg['glimepiride_tf'] = 1
df_zg['glimepiride_tf'][df_zg["glimepiride"] == "No"] = 0
df_zg['acetohexamide_tf'] = 1
df_zg['acetohexamide_tf'][df_zg["acetohexamide"] == "No"] = 0
df_zg['glipizide_tf'] = 1
df_zg['glipizide_tf'][df_zg["glipizide"] == "No"] = 0
df_zg['glyburide_tf'] = 1
df_zg['glyburide_tf'][df_zg["glyburide"] == "No"] = 0
df_zg['tolbutamide_tf'] = 1
df_zg['tolbutamide_tf'][df_zg["tolbutamide"] == "No"] = 0

### Make a new column containing the number of meds a person is taking then delete the 1/0 meds columns

In [13]:
medCount = ['metformin_tf','repaglinide_tf','nateglinide_tf','chlorpropamide_tf',
                  'glimepiride_tf','acetohexamide_tf','glipizide_tf','glyburide_tf',
                  'tolbutamide_tf','pioglitazone_tf','rosiglitazone_tf','acarbose_tf',
                  'miglitol_tf','troglitazone_tf', 'tolazamide_tf','insulin_tf',
                  'glyburide-metformin_tf','glipizide-metformin_tf',
                  'glimepiride-pioglitazone','metformin-rosiglitazone_tf',
                  'metformin-pioglitazone_tf',"glimepiride-pioglitazone_tf",
                  'metformin-rosiglitazone_tf','metformin-pioglitazone_tf']
df_zg["medication_count"] = np.nan
df_zg["medication_count"] = df_zg[medCount].sum(axis=1)
df_zg.drop(medCount, axis=1, inplace = True)

### Setting the object types to the correct ones

In [14]:
df_zg['encounter_id'] = df_zg['encounter_id'].astype(object)
df_zg['patient_nbr'] = df_zg['patient_nbr'].astype(object)
df_zg['admission_type_id'] = df_zg['admission_type_id'].astype(object)
df_zg['discharge_disposition_id'] = df_zg['discharge_disposition_id'].astype(object)
df_zg['admission_source_id'] = df_zg['admission_source_id'].astype(object)
df_zg['race'] = df_zg['race'].astype(object)
df_zg["A1Cresult"] = df_zg["A1Cresult"].astype(object)
df_zg["age"] = df_zg["age"].astype(object)
df_zg['max_glu_serum'] = df_zg['max_glu_serum'].astype(object)
df_zg['gender'] = df_zg['gender'].astype(object)
df_zg['diabetesMed'] = df_zg['diabetesMed'].astype(object)
df_zg['change'] = df_zg['change'].astype(object)

### Makes a new column that indicates what the admission_source_id means

In [15]:
df_zg['admission_source'] = np.nan
df_zg.loc[df_zg['admission_source_id'] == 1, "admission_source"] = "Physician Referral"
df_zg.loc[df_zg['admission_source_id'] == 2, "admission_source"] = "Clinic Referral"
df_zg.loc[df_zg['admission_source_id'] == 3, "admission_source"] = "HMO Referral"
df_zg.loc[df_zg['admission_source_id'] == 4, "admission_source"] = "Transfer from a hospital"
df_zg.loc[df_zg['admission_source_id'] == 5, 'admission_source'] = "Transfer from a Skilled Nursing Facility (SNF)"
df_zg.loc[df_zg['admission_source_id'] == 6, 'admission_source'] = "Transfer from another health care facility"
df_zg.loc[df_zg['admission_source_id'] == 7, 'admission_source'] = "Emergency Room"
df_zg.loc[df_zg['admission_source_id'] == 8, 'admission_source'] = "Court/Law Enforcement"
df_zg.loc[df_zg['admission_source_id'] == 9, 'admission_source'] = "Not Available"
df_zg.loc[df_zg['admission_source_id'] == 10, 'admission_source'] = "Transfer from critical access hospital"
df_zg.loc[df_zg['admission_source_id'] == 11, 'admission_source'] = "Normal Delivery"
df_zg.loc[df_zg['admission_source_id'] == 12, 'admission_source'] = "Premature Delivery"
df_zg.loc[df_zg['admission_source_id'] == 13, 'admission_source'] = "Sick Baby"
df_zg.loc[df_zg['admission_source_id'] == 14, 'admission_source'] = "Extramural Birth"
df_zg.loc[df_zg['admission_source_id'] == 15, 'admission_source'] = "Not Available"
df_zg.loc[df_zg['admission_source_id'] == 17, 'admission_source'] = "NULL"
df_zg.loc[df_zg['admission_source_id'] == 18, 'admission_source'] = "Transfer From Another Home Health Agency"
df_zg.loc[df_zg['admission_source_id'] == 19, 'admission_source'] = "Readmission to Same Home Health Agency"
df_zg.loc[df_zg['admission_source_id'] == 20, 'admission_source'] = "Not Mapped"
df_zg.loc[df_zg['admission_source_id'] == 21, 'admission_source'] = "Unknown/Invalid"
df_zg.loc[df_zg['admission_source_id'] == 22, 'admission_source'] = "Transfer from hospital inpt/same fac reslt in a sep claim"
df_zg.loc[df_zg['admission_source_id'] == 23, 'admission_source'] = "Born inside this hospital"
df_zg.loc[df_zg['admission_source_id'] == 24, 'admission_source'] = "Born outside this hospital"
df_zg.loc[df_zg['admission_source_id'] == 25, 'admission_source'] = "Transfer from Ambulatory Surgery Center"
df_zg.loc[df_zg['admission_source_id'] == 26, 'admission_source'] = "Transfer from Hospice"

### Makes a new column that indicates what the discharge_disposition_id means

In [16]:
df_zg['discharge_disposition'] = np.nan
df_zg.loc[df_zg['discharge_disposition_id'] == 1, 'discharge_disposition'] = "Discharged to home"
df_zg.loc[df_zg['discharge_disposition_id'] == 2, 'discharge_disposition'] = "Discharged/transferred to another short term hospital"
df_zg.loc[df_zg['discharge_disposition_id'] == 3, 'discharge_disposition'] = "Discharged/transferred to SNF"
df_zg.loc[df_zg['discharge_disposition_id'] == 4, 'discharge_disposition'] = "Discharged/transferred to ICF"
df_zg.loc[df_zg['discharge_disposition_id'] == 5, 'discharge_disposition'] = "Discharged/transferred to another type of inpatient care institution"
df_zg.loc[df_zg['discharge_disposition_id'] == 6, 'discharge_disposition'] = "Discharged/transferred to home with home health service"
df_zg.loc[df_zg['discharge_disposition_id'] == 7, 'discharge_disposition'] = "Left AMA"
df_zg.loc[df_zg['discharge_disposition_id'] == 8, 'discharge_disposition'] = "Discharged/transferred to home under care of Home IV provider"
df_zg.loc[df_zg['discharge_disposition_id'] == 9, 'discharge_disposition'] = "Admitted as an inpatient to this hospital"
df_zg.loc[df_zg['discharge_disposition_id'] == 10, 'discharge_disposition'] = "Neonate discharged to another hospital for neonatal aftercare"
df_zg.loc[df_zg['discharge_disposition_id'] == 11, 'discharge_disposition'] = "Expired"
df_zg.loc[df_zg['discharge_disposition_id'] == 12, 'discharge_disposition'] = "Still patient or expected to return for outpatient services"
df_zg.loc[df_zg['discharge_disposition_id'] == 13, 'discharge_disposition'] = "Hospice / home"
df_zg.loc[df_zg['discharge_disposition_id'] == 14, 'discharge_disposition'] = "Hospice / medical facility"
df_zg.loc[df_zg['discharge_disposition_id'] == 15, 'discharge_disposition'] = "Discharged/transferred within this institution to Medicare approved swing bed"
df_zg.loc[df_zg['discharge_disposition_id'] == 16, 'discharge_disposition'] = "Discharged/transferred/referred another institution for outpatient services"
df_zg.loc[df_zg['discharge_disposition_id'] == 17, 'discharge_disposition'] = "Discharged/transferred/referred to this institution for outpatient services"
df_zg.loc[df_zg['discharge_disposition_id'] == 18, 'discharge_disposition'] = "NULL"
df_zg.loc[df_zg['discharge_disposition_id'] == 19, 'discharge_disposition'] = "Expired at home. Medicaid only, hospice."
df_zg.loc[df_zg['discharge_disposition_id'] == 20, 'discharge_disposition'] = "Expired in a medical facility. Medicaid only, hospice."
df_zg.loc[df_zg['discharge_disposition_id'] == 21, 'discharge_disposition'] = "Expired, place unknown. Medicaid only, hospice."
df_zg.loc[df_zg['discharge_disposition_id'] == 22, 'discharge_disposition'] = "Discharged/transferred to another rehab fac including rehab units of a hospital."
df_zg.loc[df_zg['discharge_disposition_id'] == 23, 'discharge_disposition'] = "Discharged/transferred to a long term care hospital."
df_zg.loc[df_zg['discharge_disposition_id'] == 24, 'discharge_disposition'] = "Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare."
df_zg.loc[df_zg['discharge_disposition_id'] == 25, 'discharge_disposition'] = "Not Mapped"
df_zg.loc[df_zg['discharge_disposition_id'] == 26, 'discharge_disposition'] = "Unknown/Invalid"
df_zg.loc[df_zg['discharge_disposition_id'] == 27, 'discharge_disposition'] = "Discharged/transferred to a federal health care facility."
df_zg.loc[df_zg['discharge_disposition_id'] == 28, 'discharge_disposition'] = "Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital"
df_zg.loc[df_zg['discharge_disposition_id'] == 29, 'discharge_disposition'] = "Discharged/transferred to a Critical Access Hospital (CAH)."
df_zg.loc[df_zg['discharge_disposition_id'] == 30, 'discharge_disposition'] = "Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere"

### Makes a new column that indicates what the admission_type_id means

In [17]:
df_zg['admission_type'] = np.nan
df_zg.loc[df_zg['admission_type_id'] == 1, 'admission_type'] = "Emergency"
df_zg.loc[df_zg['admission_type_id'] == 2, 'admission_type'] = "Urgent"
df_zg.loc[df_zg['admission_type_id'] == 3, 'admission_type'] = "Elective"
df_zg.loc[df_zg['admission_type_id'] == 4, 'admission_type'] = "Newborn"
df_zg.loc[df_zg['admission_type_id'] == 5, 'admission_type'] = "Not Available"
df_zg.loc[df_zg['admission_type_id'] == 6, 'admission_type'] = "NULL"
df_zg.loc[df_zg['admission_type_id'] == 7, 'admission_type'] = "Trauma Center"
df_zg.loc[df_zg['admission_type_id'] == 8, 'admission_type'] = "Not Mapped"

### Make a new column that indicates if the amount of medication increased

In [18]:
df_zg['meds_increased'] = 0
df_zg['meds_increased'][(df_zg['metformin'] == "Up") | (df_zg['repaglinide'] == "Up") | 
                      (df_zg['nateglinide'] == "Up") | (df_zg['metformin-rosiglitazone'] == "Up") |
                      (df_zg['metformin-pioglitazone'] == "Up") | (df_zg['chlorpropamide'] == "Up") |
                      (df_zg['glimepiride'] == "Up") | (df_zg['acetohexamide'] == "Up") | 
                      (df_zg['glipizide'] == "Up") | (df_zg['glyburide'] == "Up") | 
                      (df_zg['tolbutamide'] == "Up") | (df_zg['pioglitazone'] == "Up") | 
                      (df_zg['rosiglitazone'] == "Up") | (df_zg['acarbose'] == "Up") | 
                      (df_zg['miglitol'] == "Up") | (df_zg['troglitazone'] == "Up") | 
                      (df_zg['tolazamide'] == "Up") | (df_zg['insulin'] == "Up") | 
                      (df_zg['glyburide-metformin'] == "Up")|(df_zg['glipizide-metformin'] == "Up")] = 1
df_zg['meds_increased'] = df_zg['meds_increased'].astype(object)

In [19]:
pandas_profiling.ProfileReport(df_zg)

Number of variables,51
Number of observations,101766
Total Missing (%),0.0%
Total size in memory,39.6 MiB
Average record size in memory,408.0 B
Numeric,14
Categorical,35
Boolean,2
Date,0
Text (Unique),0
Rejected,0


In [20]:
df_zg.to_csv("Diabetes_tmp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File
